In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install cityscapesscripts

In [ ]:
import torch
from torchvision import datasets
import matplotlib.pyplot as plt

## Dataset Download

In [ ]:
'''
from cityscapesscripts.download import downloader

## providing credentials and looking at packages
session = downloader.login()
downloader.get_available_packages(session=session)

## Getting
# 1. 'gtFine_trainvaltest.zip' (contains labels)
# 2. 'leftImg8bit_trainvaltest.zip' (contains images)
datalist = ['gtFine_trainvaltest.zip','leftImg8bit_trainvaltest.zip']

downloader.download_packages(session = session,
                             package_names = datalist,
                             destination_path='/content/drive/MyDrive/Colab Notebooks/MA_506') ### Need to change for your file directory
## cityscape dataset is 11.0+GB

## unzipping
!unzip '/content/drive/MyDrive/Colab Notebooks/MA_506/gtFine_trainvaltest.zip' -d '/content/drive/MyDrive/Colab Notebooks/MA_506/Cityscapes/'
!unzip '/content/drive/MyDrive/Colab Notebooks/MA_506/leftImg8bit_trainvaltest.zip' -d '/content/drive/MyDrive/Colab Notebooks/MA_506/Cityscapes/'
'''

## Data Loader

## Define Model

In [ ]:
from torchvision.models.segmentation import fcn_resnet50

model = fcn_resnet50(pretrained=True) # The patches are created to disrupt the resnet 50 model.
model.eval()

In [ ]:
import torch
from torchvision import transforms, datasets
from torchvision.utils import save_image
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from PIL import Image
import os

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define paths for images, labels, and saving
image_dir = "/content/drive/MyDrive/Colab Notebooks/MA_506/Cityscapes/Robel_Rotate/Clean Images"
label_dir = "/content/drive/MyDrive/Colab Notebooks/MA_506/Cityscapes/Robel_Rotate/Labels"
save_dir = '/content/drive/MyDrive/Colab Notebooks/MA_506/Cityscapes/Dirty Images'
os.makedirs(save_dir, exist_ok=True)

# Transformations
image_transform = transforms.Compose([
    transforms.Resize((128, 256)),
    transforms.ToTensor(),
])

label_transform = transforms.Compose([
    transforms.Resize((128, 256)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.long()),
])

# Custom Dataset
class CityscapesDataset(Dataset):
    def __init__(self, image_dir, label_dir, image_transform=None, label_transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.image_transform = image_transform
        self.label_transform = label_transform
        self.image_filenames = sorted(os.listdir(image_dir))
        self.label_filenames = sorted(os.listdir(label_dir))

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        # Load image and label
        image_path = os.path.join(self.image_dir, self.image_filenames[idx])
        label_path = os.path.join(self.label_dir, self.label_filenames[idx])

        image = Image.open(image_path).convert("RGB")
        label = Image.open(label_path)

        # Apply transformations
        if self.image_transform:
            image = self.image_transform(image)
        if self.label_transform:
            label = self.label_transform(label)

        # Get the filename
        filename = os.path.basename(image_path)

        return image, label, filename

# Create dataset and DataLoader
dataset = CityscapesDataset(
    image_dir=image_dir,
    label_dir=label_dir,
    image_transform=image_transform,
    label_transform=label_transform
)
batch_size = 1
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Loop through DataLoader and save perturbed images
epsilon = 0.5  # Perturbation strength
for batch_idx, (images, labels, filenames) in enumerate(data_loader):
    images, labels = images.to(device), labels.to(device)
    images.requires_grad = True

    # Forward pass through the model
    output = model(images)['out']
    loss = F.cross_entropy(output, labels.squeeze(1))
    model.zero_grad()

    # Backward pass to calculate gradients
    loss.backward()

    # Generate perturbed images
    data_grad = images.grad.data
    perturbed_images = torch.stack(
        [fgsm_attack_patch(images[i], epsilon, data_grad[i]) for i in range(images.size(0))]
    )

    # Save each perturbed image
    for i in range(perturbed_images.size(0)):
        original_filename = filenames[i]
        save_path = os.path.join(save_dir, f"corrupt_50_{original_filename}")
        save_image(perturbed_images[i], save_path)
        print(f"Saved perturbed image: {save_path}")